In [1]:
pip install codecarbon

  Using cached codecarbon-2.3.5-py3-none-any.whl.metadata (6.6 kB)
  Using cached pynvml-11.5.0-py3-none-any.whl.metadata (7.8 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached rapidfuzz-3.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached codecarbon-2.3.5-py3-none-any.whl (174 kB)
Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
Using cached pynvml-11.5.0-py3-none-any.whl (53 kB)
Using cached rapidfuzz-3.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import time
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import pandas as pd
from codecarbon import OfflineEmissionsTracker

2024-04-24 12:14:41.750165: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-24 12:14:58.386300: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load li

In [3]:
training_set=tf.data.Dataset.load('tensorflow_datasets/training_set')
validation_set=tf.data.Dataset.load('tensorflow_datasets/validation_set')
test_set=tf.data.Dataset.load('tensorflow_datasets/test_set')

training_class_counts = {}
validation_class_counts = {}
test_class_counts = {}

for images, labels in training_set:
    for label in labels.numpy():
        label = label.item() 
        if label in training_class_counts:
            training_class_counts[label] += 1
        else:
            training_class_counts[label] = 1

for images, labels in validation_set:
    for label in labels.numpy():
        label = label.item()  
        if label in validation_class_counts:
            validation_class_counts[label] += 1
        else:
            validation_class_counts[label] = 1

for images, labels in test_set:
    for label in labels.numpy():
        label = label.item() 
        if label in test_class_counts:
            test_class_counts[label] += 1
        else:
            test_class_counts[label] = 1


In [4]:

print("Training set labels:")
for label, count in training_class_counts.items():
    print("{}".format(label))

print("Training set:")
for label, count in training_class_counts.items():
    print("{}".format(count))

print("\nValidation set labels:")
for label, count in validation_class_counts.items():
    print("{}".format(label))

print("\nValidation set:")
for label, count in validation_class_counts.items():
    print("{}".format(count))

print("\nTest set labels:")
for label, count in test_class_counts.items():
    print("{}".format(label))

print("\nTest set:")
for label, count in test_class_counts.items():
    print("{}".format(count))

Training set labels:
25
100
66
17
72
95
43
26
93
88
23
11
90
28
59
64
68
55
69
81
78
70
6
14
12
19
27
3
76
22
56
75
42
74
5
94
7
33
38
73
34
35
18
80
29
97
84
60
82
63
2
86
65
15
87
20
47
50
4
46
45
89
21
0
36
52
49
61
57
24
16
99
85
54
48
71
40
79
83
1
96
92
101
10
77
91
32
41
98
8
62
51
53
67
31
58
39
30
13
37
44
9
Training set:
27
46
30
62
166
70
77
26
143
158
31
73
60
61
90
86
41
86
46
94
27
64
27
36
35
43
49
44
217
74
53
89
116
101
30
107
69
29
29
151
33
57
34
137
67
68
46
32
106
38
29
50
49
31
131
28
57
228
53
54
167
64
46
28
89
73
76
45
97
29
66
33
46
54
37
76
106
88
70
47
49
32
38
70
118
50
33
44
51
33
41
68
48
41
35
52
51
36
32
42
28
32

Validation set labels:
44
8
65
93
92
5
40
85
63
51
46
39
38
21
27
47
99
75
101
37
96
69
11
72
48
74
14
55
57
79
42
76
41
36
54
32
71
94
1
91
53
80
70
50
56
43
34
26
31
3
2
60
81
16
12
78
13
23
62
82
18
68
17
29
49
4
24
59
77
90
100
15
19
6
45
33
35
95
30
84
67
83
97
87
58
52
20
9
28
98
0
22
61
86
88
10
89
64
7
73
66
25

Validation set:
9
9
6
8

In [5]:


num_training_examples = 0
num_validation_examples = 0
num_test_examples = 0

for example in training_set:
  num_training_examples += 1

for example in validation_set:
  num_validation_examples += 1

for example in test_set:
  num_test_examples += 1

print('Total Number of Training Images: {}'.format(num_training_examples))
print('Total Number of Validation Images: {}'.format(num_validation_examples))
print('Total Number of Test Images: {} \n'.format(num_test_examples))

num_classes = 102
print('Total Number of Classes: {}'.format(num_classes))


# In[4]:


num_training_batches = tf.data.experimental.cardinality(training_set).numpy()
num_validation_batches = tf.data.experimental.cardinality(validation_set).numpy()
num_test_batches = tf.data.experimental.cardinality(test_set).numpy()

print("Number of batches in the training set:", num_training_batches)
print("Number of batches in the validation set:", num_validation_batches)
print("Number of batches in the test set:", num_test_batches)

Total Number of Training Images: 205
Total Number of Validation Images: 26
Total Number of Test Images: 26 



NameError: name 'dataset_info' is not defined

In [6]:
input_shape = (224, 224, 3)
model = tf.keras.models.load_model('gs://plant_buddy/models/models/model12_8conv_17', compile=False)

# model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy']) 
base_learning_rate = 0.001
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate), metrics=['accuracy']) 

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      832       
                                                                 
 activation (Activation)     (None, 224, 224, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 128)     32896     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                  

In [5]:
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.layers import Activation, Dense, Flatten, Dropout, Reshape
from tensorflow.keras import layers, Sequential

input_shape = (224, 224, 3)

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.4),
])


def tensorflow_based_model():
    model = Sequential() 
    
    model.add(data_augmentation) 
    
    model.add(Conv2D(filters=64, kernel_size=2, input_shape=(224,224,3), padding='same')) 
    model.add(Activation('relu')) 
    model.add(MaxPooling2D(pool_size=2)) 

#     model.add(Conv2D(filters=32, kernel_size=2, activation='relu', padding='same')) 
#     model.add(MaxPooling2D(pool_size=2))  
    
#     model.add(Conv2D(filters=64, kernel_size=2, activation='relu', padding='same')) 
#     model.add(MaxPooling2D(pool_size=2))  
    
    model.add(Conv2D(filters=128, kernel_size=2, activation='relu', padding='same')) 
    model.add(MaxPooling2D(pool_size=2))  
    
    model.add(Conv2D(filters=128, kernel_size=2, activation='relu', padding='same')) 
    model.add(MaxPooling2D(pool_size=2))  
    
    model.add(Conv2D(filters=256, kernel_size=2, activation='relu', padding='same'))  
    model.add(MaxPooling2D(pool_size=2))  
    
    model.add(Conv2D(filters=256, kernel_size=2, activation='relu', padding='same')) 
    model.add(MaxPooling2D(pool_size=2)) 
    
    model.add(Conv2D(filters=512, kernel_size=2, activation='relu', padding='same')) 
    model.add(MaxPooling2D(pool_size=2)) 
        
    model.add(Conv2D(filters=512, kernel_size=2, activation='relu', padding='same'))  
    model.add(MaxPooling2D(pool_size=2)) 

    model.add(Dropout(0.2))  
    model.add(Flatten())  
    model.add(Dense(150))  
    model.add(Activation('relu'))  
    model.add(Dropout(0.2))  
    model.add(Dense(102, activation='softmax'))  

    return model

In [6]:
model = tensorflow_based_model()
model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy']) 

In [ ]:
tracker = OfflineEmissionsTracker(country_iso_code="USA", measure_power_secs=120, output_dir="gs://plant_buddy/energy", output_file="emissions12_8conv1_apr.csv")
tracker.start()

history = model.fit(training_set,
        batch_size = 32,
        epochs=70,
        validation_data=validation_set)

emissions: float = tracker.stop()
print(emissions)

model.save('gs://plant_buddy/models/models/model12_8conv1_apr')
pd.DataFrame.from_dict(history.history).to_csv('gs://plant_buddy/histories/history12_8conv1_apr.csv',index=False)

[codecarbon INFO @ 12:22:58] offline tracker init
[codecarbon INFO @ 12:22:58] [setup] RAM Tracking...
[codecarbon INFO @ 12:22:58] [setup] GPU Tracking...
[codecarbon INFO @ 12:22:58] No GPU found.
[codecarbon INFO @ 12:22:58] [setup] CPU Tracking...
[codecarbon WARNING @ 12:22:58] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 12:23:00] We saw that you have a Intel(R) Xeon(R) CPU @ 2.30GHz but we don't know it. Please contact us.
[codecarbon INFO @ 12:23:00] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.30GHz
[codecarbon INFO @ 12:23:00] >>> Tracker's metadata:
[codecarbon INFO @ 12:23:00]   Platform system: Linux-5.10.0-28-cloud-amd64-x86_64-with-glibc2.35
[codecarbon INFO @ 12:23:00]   Python version: 3.10.13
[codecarbon INFO @ 12:23:00]   CodeCarbon version: 2.3.5
[codecarbon INFO @ 12:23:00]   Available RAM : 29.390 GB
[codecarbon INFO @ 12:23:00]   CPU count: 8
[codecarbon INFO @ 12:23:00]   CPU model: Intel(R) Xeon(R) CPU

Epoch 1/70
 78/205 [==========>...................] - ETA: 3:03 - loss: 4.5565 - accuracy: 0.0317

[codecarbon INFO @ 12:25:00] Energy consumed for RAM : 0.000367 kWh. RAM Power : 11.0212082862854 W
[codecarbon INFO @ 12:25:00] Energy consumed for all CPUs : 0.001417 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:25:00] 0.001784 kWh of electricity used since the beginning.


163/205 [======================>.......] - ETA: 59s - loss: 4.5197 - accuracy: 0.0301 

[codecarbon INFO @ 12:27:00] Energy consumed for RAM : 0.000735 kWh. RAM Power : 11.0212082862854 W
[codecarbon INFO @ 12:27:00] Energy consumed for all CPUs : 0.002834 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:27:00] 0.003568 kWh of electricity used since the beginning.


...

[codecarbon INFO @ 18:07:26] Energy consumed for RAM : 0.063262 kWh. RAM Power : 11.0212082862854 W
[codecarbon INFO @ 18:07:26] Energy consumed for all CPUs : 0.243960 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:07:26] 0.307222 kWh of electricity used since the beginning.


0.11308907500782303


2024-04-24 18:07:28.679887: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'random_flip_input' with dtype float and shape [?,224,224,3]
	 [[{{node random_flip_input}}]]
2024-04-24 18:07:28.798682: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,224,224,3]
	 [[{{node inputs}}]]
2024-04-24 18:07:28.812470: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'random_flip_input' with dtype float and shape [

INFO:tensorflow:Assets written to: gs://plant_buddy/models/models/model12_8conv1_apr/assets


INFO:tensorflow:Assets written to: gs://plant_buddy/models/models/model12_8conv1_apr/assets


In [ ]:
tracker = OfflineEmissionsTracker(country_iso_code="USA", measure_power_secs=30, output_dir="gs://plant_buddy/energy", output_file="emissions12_8conv_17_re_eval.csv")
tracker.start()

loss, accuracy = model.evaluate(test_set)
print('Test accuracy :', accuracy)
print('Test loss :', loss)

emissions: float = tracker.stop()
print(emissions)

In [ ]:
# converter = tf.lite.TFLiteConverter.from_saved_model('effnet2_apr')
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tflite_quant_model = converter.convert()

converter = tf.lite.TFLiteConverter.from_saved_model('gs://plant_buddy/models/models/model12_9conv17_re2_ad1')
tflite_model = converter.convert()

with open('model12_9conv17_re2_ad1.tflite', 'wb') as f:
  f.write(tflite_model)